# Ayudantía 1

MAN 3160 - Sistemas Recomendadores


## Importar Librerías

In [1]:
import numpy as np
import json
import requests
import heapq
import math
import matplotlib.pyplot as plt
from sklearn.metrics import pairwise_distances
from sklearn.decomposition import PCA
from io import BytesIO
import pickle
import pandas as pd
import time
import scipy.sparse as sparse
import implicit

C:\Users\alfi9\AppData\Local\Programs\Python\Python39\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Descargamos datos que vienen previamente calculados: 
- transacciones/interacciones de cada usuario 
- transaciones para evaluar el modelo 
- embeddings de descripciones calculados con BERT  
- embeddings de descripciones calculados con BERT-large
- datos de libros con información de titulo, descripcion, año de publicacion, entre otros. 

In [2]:
import urllib.request
urllib.request.urlretrieve("https://www.dropbox.com/s/57tel5zqopkssrh/books.csv?dl=1", "books.csv")
urllib.request.urlretrieve("https://www.dropbox.com/s/zpnnoy1i8ljf9fg/goodreads_bert_embeddings.npy?dl=1", "goodreads_bert_embeddings.npy")
urllib.request.urlretrieve("https://www.dropbox.com/s/a8hcc9w30y7r3jl/goodreads_bert_large_embeddings.npy?dl=1", "goodreads_bert_large_embeddings.npy")
urllib.request.urlretrieve("https://www.dropbox.com/s/dqeqpsr0vdvmcy0/goodreads_past_interactions.json?dl=1", "goodreads_past_interactions.json")
urllib.request.urlretrieve("https://www.dropbox.com/s/rjtzhmb2zbpp30q/goodreads_test_interactions.json?dl=1", "goodreads_test_interactions.json")

('goodreads_test_interactions.json',
 <http.client.HTTPMessage at 0x2098b25d3a0>)

# Cargar datos adicionales

In [3]:
df_books = pd.read_csv('books.csv', sep=',')
df_books.head()

,book_id,goodreads_book_id,best_book_id,work_id,books_count,isbn,isbn13,authors,original_publication_year,original_title,...,work_ratings_count,work_text_reviews_count,ratings_1,ratings_2,ratings_3,ratings_4,ratings_5,image_url,small_image_url,book_desc
0,1,2767052,2767052,2792775,272,439023483,9.780439e+12,Suzanne Collins,2008.0,The Hunger Games,...,4942365,155254,66715,127936,560092,1481305,2706317,https://images.gr-assets.com/books/1447303603m...,https://images.gr-assets.com/books/1447303603s...,Winning will make you famous. Losing means cer...
1,2,3,3,4640799,491,439554934,9.780440e+12,"J.K. Rowling, Mary GrandPré",1997.0,Harry Potter and the Philosopher's Stone,...,4800065,75867,75504,101676,455024,1156318,3011543,https://images.gr-assets.com/books/1474154022m...,https://images.gr-assets.com/books/1474154022s...,Harry Potter's life is miserable. His parents ...
2,3,41865,41865,3212258,226,316015849,9.780316e+12,Stephenie Meyer,2005.0,Twilight,...,3916824,95009,456191,436802,793319,875073,1355439,https://images.gr-assets.com/books/1361039443m...,https://images.gr-assets.com/books/1361039443s...,About three things I was absolutely positive.F...
3,4,2657,2657,3275794,487,61120081,9.780061e+12,Harper Lee,1960.0,To Kill a Mockingbird,...,3340896,72586,60427,117415,446835,1001952,1714267,https://images.gr-assets.com/books/1361975680m...,https://images.gr-assets.com/books/1361975680s...,The unforgettable novel of a childhood in a sl...
4,5,4671,4671,245494,1356,743273567,9.780743e+12,F. Scott Fitzgerald,1925.0,The Great Gatsby,...,2773745,51992,86236,197621,606158,936012,947718,https://images.gr-assets.com/books/1490528560m...,https://images.gr-assets.com/books/1490528560s...,Alternate Cover Edition ISBN: 0743273567 (ISBN...


In [4]:
# diccionario con id del usuario y id de libros con los que ha interactuado en el pasado 
with open('goodreads_past_interactions.json') as f:
    user_interactions = json.load(f)

# diccionario con id del usuario y id de libros para testear el modelo     
with open('goodreads_test_interactions.json') as f:
    user_interactions_test = json.load(f)

In [5]:
user_interactions['1']

[258,
 268,
 3638,
 1796,
 867,
 2738,
 4691,
 916,
 11,
 3889,
 136,
 6665,
 35,
 60,
 148,
 10,
 4,
 57,
 1521,
 70,
 103,
 36,
 119,
 13,
 66,
 2002,
 43,
 287,
 1041,
 67,
 46,
 22,
 115,
 31,
 16,
 256,
 273,
 378,
 329,
 98,
 216,
 1176,
 140,
 1310,
 414,
 85,
 219,
 177,
 102,
 95,
 225,
 76,
 100,
 171,
 485,
 325,
 498,
 323,
 72,
 496,
 1030,
 1055,
 2770,
 1187,
 2535,
 3294,
 4893,
 2133,
 262,
 437,
 421,
 901,
 212]

In [6]:
# dict index 2 book id and vice-versa for recommendation 
idx2bookid = {i: id_ for i, id_ in enumerate(df_books.book_id)}
bookid2idx = {id_:i for i, id_ in enumerate(df_books.book_id)}

# Cargar características pre-entrenadas: BERT y BERT-large

En esta sección se trabajará con modelos pre-entrenados de modelos de lenguage BERT y BERT-large que convierten texto a embeddings. 

Bidirectional Encoder Representations from Transformers (BERT) es una técnica de NLP (Natural Language Processing) desarrollada por Google y publicada en 2018 por Jacob Devlin. 

Actualmente Google utiliza BERT para entender las consultas de los usuarios en su buscador. 

Tiene dos versiones: 
- **BERT:** 12 capas, 12 cabezales de atencion y 110 millones de parámetros. Genera vectores de 768 dimensiones 
- **BERT-large:** 24 capas, 16 cabezales de atencion y 340 millones de parámetros.  

![BERT y BERT-large](http://jalammar.github.io/images/bert-base-bert-large.png)

![BERT y BERT-large arquitectura](http://jalammar.github.io/images/bert-base-bert-large-encoders.png)

En este caso los textos que utilizaremos son los títulos de los libros con su descripción y compararemos los resultados de recomendación con BERT y BERT-large. Para efectos de este trabajo los vectores de características ya fueron entrenados y guardados en archivos numpy. A continuación son cargados en memoria.

Para mayores detalles sobre el modelo de lenguaje BERT se recomienda revisar el siguiente artículo:
- [BERT: Pre-training of Deep Bidirectional Transformers for Language Understanding](https://arxiv.org/pdf/1810.04805.pdf)

In [7]:
bert_featmat = np.load('goodreads_bert_embeddings.npy', allow_pickle=True)
bert_large_featmat = np.load('goodreads_bert_large_embeddings.npy', allow_pickle=True)

In [8]:
bert_featmat.shape

(4287, 768)

In [9]:
bert_large_featmat.shape

(4287, 1024)

In [10]:
bert_featmat[5]

array([ 4.84995842e-01,  3.47123146e-02,  1.82838421e-02,  4.71778587e-02,
       -9.59259123e-02,  4.44397539e-01, -1.40853271e-01, -1.73830017e-02,
       -3.44723985e-02,  5.03976084e-02, -1.25274509e-01,  2.33933926e-01,
        1.72393486e-01, -4.44924459e-02,  6.76755458e-02, -9.40277800e-02,
        7.49101043e-02,  4.60708678e-01, -2.49723822e-01, -1.68240890e-01,
       -9.21016037e-02, -6.15491830e-02,  9.05511603e-02,  3.28459591e-02,
       -4.04576123e-01, -1.64132327e-01,  1.70409709e-01,  2.54194383e-02,
        1.03816532e-01, -3.57919652e-03,  7.22740144e-02, -2.62293249e-01,
        4.91619343e-04, -2.49186531e-02,  1.27423316e-01,  5.59403673e-02,
        1.01800971e-01,  4.02531624e-02, -2.25695863e-01, -4.02649213e-03,
       -3.45016539e-01,  1.14103138e-01, -1.20682888e-01,  1.04268208e-01,
       -3.60891335e-02,  1.34674117e-01,  5.88834062e-02,  2.11443812e-01,
       -9.84561443e-02, -1.35545731e-01,  1.21868595e-01, -1.37343898e-01,
       -7.76119754e-02, -

## Reducción de dimensionalidad (PCA)

In [11]:
# Project into a 20 PCA feature space
pca20_bert_featmat = PCA(n_components=20).fit_transform(bert_featmat)
pca20_bert_large_featmat = PCA(n_components=20).fit_transform(bert_large_featmat)

In [12]:
pca20_bert_featmat.shape

(4287, 20)

# Similar document retrieval 

En esta sección utilizaremos los vectores cargados para hacer un sistema de recuperación o búsqueda de información, para diferentes métricas de distancia.

Buscamos libros similares de acuerdo a la representación vectorial (BERT) de su título y descripción. 


In [13]:
# format results 
pd.options.display.max_colwidth = 50
pd.set_option('display.max_colwidth', -1)

C:\Users\alfi9\AppData\Local\Temp\ipykernel_47272\3993557653.py:3: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1)


In [14]:
# Find similar images by image id
def find_similar_books(embedding, query_id=None, metric='euclidean', topk=10):
    
    n = embedding.shape[0]
    
    if query_id is None:
        query_i = np.random.randint(n)
        query_id = idx2bookid[query_i]
    
    else:
        query_i = bookid2idx[query_id]
        
    
    distances = pairwise_distances(embedding[query_i].reshape(1,-1), embedding, metric=metric)
    heap = []
    for i in range(n):            
        if len(heap) < topk:
            heapq.heappush(heap, (-distances[0][i], i))
        else:
            heapq.heappushpop(heap, (-distances[0][i], i))

    heap.sort(reverse=True)
    rec_ids = [idx2bookid[i] for _,i in heap]
    
    return rec_ids

In [15]:
# libros similares al libro de id 41865 (Twilight) utilizando distancia euclideana. se puede cambiar a "cosine" 
similar_books = find_similar_books(bert_featmat, query_id = 3, metric = 'euclidean', topk=10 )
similar_books

[3, 2908, 3115, 2303, 7334, 7235, 5721, 3510, 9552, 9696]

In [16]:
df_books[df_books.book_id.isin(similar_books)][['book_id', 'original_title', 'book_desc', 'authors']]

,book_id,original_title,book_desc,authors
2,3,Twilight,"About three things I was absolutely positive.First, Edward was a vampire.Second, there was a part of him—and I didn't know how dominant that part might be—that thirsted for my blood.And third, I was unconditionally and irrevocably in love with him.In the first book of the Twilight Saga, internationally bestselling author Stephenie Meyer introduces Bella Swan and Edward Cullen, a pair of star-crossed lovers whose forbidden relationship ripens against the backdrop of small-town suspicion and a mysterious coven of vampires. This is a love story with bite.",Stephenie Meyer
1423,2303,Bloody Bones,"In Laurell K. Hamilton's ""New York Times"" bestselling novels, Anita Blake, vampire hunter and animator, takes a bite out of crime-of the supernatural kind. But even someone who deals with death on a daily basis can be unnerved by its power... When Branson, Missouri, is hit with a death wave-four unsolved murders-it doesn't take an expert to realize that all is not well. But luckily for the locals, Anita is an expert-in just the kinds of preternatural goings-on that have everyone spooked. And she's got an ""in"" with just the kind of creature who can make sense of the slayings: a sexy master vampire known as Jean Claude.",Laurell K. Hamilton
1708,2908,"Severed Heads, Broken Hearts","Robyn Schneider's The Beginning of Everything is a witty and heart-wrenching teen novel that will appeal to fans of books by John Green and Ned Vizzini, novels such as The Perks of Being a Wallflower, and classics like The Great Gatsby and The Catcher in the Rye.Varsity tennis captain Ezra Faulkner was supposed to be homecoming king, but that was before—before his girlfriend cheated on him, before a car accident shattered his leg, and before he fell in love with unpredictable new girl Cassidy Thorpe.As Kirkus Reviews said in a starred review, ""Schneider takes familiar stereotypes and infuses them with plenty of depth. Here are teens who could easily trade barbs and double entendres with the characters that fill John Green's novels.""Funny, smart, and including everything from flash mobs to blanket forts to a poodle who just might be the reincarnation of Jay Gatsby, The Beginning of Everything is a refreshing contemporary twist on the classic coming-of-age novel—a heart-wrenching story about how difficult it is to play the part that people expect, and how new beginnings can stem from abrupt and tragic endings.",Robyn Schneider
1808,3115,A Hunger Like No Other,"In New York Times and USA TODAY bestselling author Kresley Cole’s sizzling series, a fierce werewolf and a bewitching vampire become unlikely soul mates whose passion will test the boundaries of life and death.After enduring years of torture from the vampire horde, Lachlain MacRieve, leader of the Lykae Clan, is enraged to find the predestined mate he’s waited millennia for is a vampire. Or partly one. Emmaline Troy is a small, ethereal half Valkyrie/half vampire, who somehow begins to soothe the fury burning within him.Sheltered Emmaline finally sets out to uncover the truth about her deceased parents—until a powerful Lykae claims her as his mate and forces her back to his ancestral Scottish castle. There, her fear of the Lykae—and their notorious dark desires—ebbs as he begins a slow, wicked seduction to sate her own dark cravings.Yet when an ancient evil from her past resurfaces, will their desire deepen into a love that can bring a proud warrior to his knees and turn a gentle beauty into the fighter she was born to be?",Kresley Cole
1999,3510,"Cerulean Sins (Anita Blake, Vampire Hunter, #11)","Cerulean Sins, the eleventh entry in the hugely-popular Anita Blake series, finds everyone’s favorite vampire hunter keeping house and kicking butt.Anita Blake is trying to get her life back to “normal” after a break-up with her werewolf lover. She has settled into a pattern of domesticity, which means that the new man in her life, the leopard shapeshifte

# Recomendación basada en contenido

In [17]:
def recommend(embedding, user_id=None, topk=10, metric='cosine'):
    
    #print("user_id = ", user_id)
    
    user_id = str(user_id)
    
    #Calculate distance metrics
    trx = user_interactions[user_id]
    n = embedding.shape[0]
    distances = 1e9
    
    # recorremos transacciones pasadas del usuario 
    for t in trx:
        query_i = bookid2idx[t]
        
        # recomendamos items más cercanos a items con los que interactuó el usuario
        distances = np.minimum(distances, pairwise_distances(
                embedding[query_i].reshape(1,-1), embedding, metric=metric).reshape(-1))

    #Rank items de menor a mayor distancia (nos quedamos con los topk)
    trx_set = set(trx)
    heap = []
    for i in range(n):
        if idx2bookid[i] in trx_set:
            continue
        if len(heap) < topk:
            heapq.heappush(heap, (-distances[i], i))
        else:
            heapq.heappushpop(heap, (-distances[i], i))
    heap.sort(reverse=True)
    
    # utilizamos un heap para extraer los items ordenados de menor a mayor distancia 
    recommended_ids = [idx2bookid[i] for _,i in heap]
    
    # retornar los que el usuario no haya consumido
    filtered_recommended_ids = []
    
    return recommended_ids

In [18]:
# recomendación para el usuario id = 50101 , utilizando bert con reduccion de dimensionalidad a 20 
user_id = '50101'
rec = recommend(pca20_bert_featmat, user_id=user_id, topk=10)
rec 

[4509, 5002, 4376, 964, 2292, 7937, 5126, 9473, 5244, 390]

## Revisión de recomendación

In [19]:
df_books[df_books.book_id.isin(user_interactions['50101'])][['book_id', 'original_title', 'book_desc', 'authors']]

,book_id,original_title,book_desc,authors
2,3,Twilight,"About three things I was absolutely positive.First, Edward was a vampire.Second, there was a part of him—and I didn't know how dominant that part might be—that thirsted for my blood.And third, I was unconditionally and irrevocably in love with him.In the first book of the Twilight Saga, internationally bestselling author Stephenie Meyer introduces Bella Swan and Edward Cullen, a pair of star-crossed lovers whose forbidden relationship ripens against the backdrop of small-town suspicion and a mysterious coven of vampires. This is a love story with bite.",Stephenie Meyer
5,6,The Fault in Our Stars,"Despite the tumor-shrinking medical miracle that has bought her a few years, Hazel has never been anything but terminal, her final chapter inscribed upon diagnosis. But when a gorgeous plot twist named Augustus Waters suddenly appears at Cancer Kid Support Group, Hazel's story is about to be completely rewritten.Insightful, bold, irreverent, and raw, The Fault in Our Stars is award-winning author John Green's most ambitious and heartbreaking work yet, brilliantly exploring the funny, thrilling, and tragic business of being alive and in love.",John Green
7,8,The Catcher in the Rye,"The hero-narrator of The Catcher in the Rye is an ancient child of sixteen, a native New Yorker named Holden Caulfield. Through circumstances that tend to preclude adult, secondhand description, he leaves his prep school in Pennsylvania and goes underground in New York City for three days. The boy himself is at once too simple and too complex for us to make any final comment about him or his story. Perhaps the safest thing we can say about Holden is that he was born in the world not just strongly attracted to beauty but, almost, hopelessly impaled on it. There are many voices in this novel: children's voices, adult voices, underground voices-but Holden's voice is the most eloquent of all. Transcending his own vernacular, yet remaining marvelously faithful to it, he issues a perfectly articulated cry of mixed pain and pleasure. However, like most lovers and clowns and poets of the higher orders, he keeps most of the pain to, and for, himself. The pleasure he gives away, or sets aside, with all his heart. It is there for the reader who can handle it to keep. J.D. Salinger's classic novel of teenage angst and rebellion was first published in 1951. The novel was included on Time's 2005 list of the 100 best English-language novels written since 1923. It was named by Modern Library and its readers as one of the 100 best English-language novels of the 20th century. It has been frequently challenged in the court for its liberal use of profanity and portrayal of sexuality and in the 1950's and 60's it was the novel that every teenage boy wants to read.",J.D. Salinger
8,10,Pride and Prejudice,"«È cosa ormai risaputa che a uno scapolo in possesso di un'ingente fortuna manchi soltanto una moglie. Questa verità è cosí radicata nella mente delle famiglie del luoho che, nel momento in cui un simile personaggio viene a far parte del vicinato, prima ancora di conoscere anche lontanamente i suoi desiderî in proposito, viene immediatamente considerato come proprietà legittima di una o l'altra delle loro figlie.»Orgoglio e pregiudizio è uno dei primi romanzi di Jane Austen. La scrittrice lo iniziò a ventun anni; il libro, rifiutato da un editore londinese, rimase in un cassetto fino alla sua pubblicazione anonima nel 1813, e da allora è considerato tra i piú importanti romanzi della letteratura inglese. È la storia delle cinque sorelle Bennet e dei loro corteggiatori, con al centro il romantico contrasto tra l'adorabile e capricciosa Elizabeth e l'altezzoso Darcy; lo spirito di osservazione implacabile e quasi cinico, lo studio arguto dei caratteri, la satira delle vanità e delle debolezze della vita domestica, fanno di questo romanzo una delle piú efficaci e indimenticabili commedie di costume del periodo Regency inglese.

# Recomendación ALS

In [20]:
user_items = {}
itemset = set()

for uid in user_interactions.keys():
    for iid in user_interactions[uid]:
        if int(uid) not in user_items:
            user_items[int(uid)] = []

        user_items[int(uid)].append(iid)
        itemset.add(iid)

itemset = np.sort(list(itemset))

sparse_matrix = np.zeros((len(user_items), len(itemset)))

for i, items in enumerate(user_items.values()):
    sparse_matrix[i] = np.isin(itemset, items, assume_unique=True).astype(int)

user_item_matrix = sparse.csr_matrix(sparse_matrix)

user_ids = {key: i for i, key in enumerate(user_items.keys())}
items_ids = {key: i for i, key in enumerate(itemset)}

In [21]:
model_als = implicit.als.AlternatingLeastSquares()
model_als.fit(user_item_matrix)

100%|██████████████████████████████████████████████████████████████████████████████████| 15/15 [00:44<00:00,  2.95s/it]


In [22]:
recommendations = model_als.recommend(userid=user_ids[965], user_items=user_item_matrix[user_ids[965]], N=10)
rec_als = [idx2bookid[r] for r in recommendations[0]]

In [23]:
rec_als = [idx2bookid[r] for r in recommendations[0]]

### Comparando recomendaciones

In [24]:
df_books[df_books.book_id.isin(rec)][['book_id', 'original_title', 'book_desc', 'authors']]

,book_id,original_title,book_desc,authors
293,390,The Strange Case of Dr Jekyll and Mr Hyde,"Un abogado, Gabriel John Utterson, investiga la extraña relación entre su viejo amigo, el doctor. Henry Jekyll, y el misántropo Edward Hyde. En realidad, Jekyll es un científico que crea una poción o bebida que tiene la capacidad de separar la parte más humana del lado más maléfico de una persona. Cuando Jekyll bebe esta mezcla se convierte en Edward Hyde, un criminal capaz de cualquier atrocidad. Desesperado por la necesidad cada vez más frecuente e incontrolable de convertirse en ese ser malvado, el doctor Jekyll decide suicidarse.","Robert Louis Stevenson, Vladimir Nabokov, Mervyn Peake, Dan Chaon"
683,964,The Hobbit and The Lord of the Rings,لجزء الثالث من ملحمة جيه أر أر تولكين الرائعة سيد الخواتم دخل أفراد رفقة الخاتم فى مغامرات منفصلة أثناء إنجاز المهمة وسيرها فى طريق التنفيذ - اما أراجون الذى كشف أنه الوريث الخلفى لملوك الغرب القدامى فقد أنضم إلى خيالة روهان ضد قوى أيزنجارد وأشترك فى نصر هورنبرج الذى تحقق بصعوبة وبأس - أما ميرى وبيبين كانا قد تعرضا للإختطاف من جانب الأوركيين فقد هربا إلى غابة فانجون وهناك قابلا الأنتيين وعاد جاندالف فيما يشبة المعجزة وهزم الساحر الشرير سارومان وفى نفس الوقت فإن سام وفرودو راحا يتقدمان بإتجاه موردو لتدمير الخاتم وبرفقتهما سميجول - جولام وهو لا يزال مهووسا ب شيئة الثمين وبعد معركة مع أنثى العنكبوت العملاق شيلوب ترك سام سيده طنا منه أنه مات ولكن فرودو كان لا يزال على قيد الحياة فى أيدى الأوركيين وكانت جيوش سيد الظلام تتجمع بإستمرار,J.R.R. Tolkien
1417,2292,ساق البامبو [Saq al-Bambu],لماذا كان جلوسي تحت الشجرة يزعج أمي؟ أتراها كانت تخشى أن تنبت لي جذور تضرب في عمق الأرض ما يجعل عودتي إلى بلاد أبي أمراً مستحيلاً؟..ربما، ولكن، حتى الجذور لا تعني شيئاً أحياناً. لو كنت مثل نبتة البامبو.. لا انتماء لها.. نقتطع جزءاً من ساقها.. نغرسه، بلا جذور، في أي أرض.. لا يلبث الساق طويلاً حتى تنبت له جذور جديدة.. تنمو من جديد.. في أرض جديدة.. بلا ماض.. بلا ذاكرة.. لا يلتفت إلى اختلاف الناس حول تسميته.. كاوايان في الفلبين.. خيزران في الكويت.. أو بامبو في أماكن أخرى.,"سعود السنعوسي, Saud Alsanousi"
2363,4376,شيكاجو,يقول الأستاذ جلال أمين عن هذه الرواية المتميزة: ها هى رواية علاء الأسوانى الجديدة «شيكاجو» تستحق بدورها نجاحًا مماثلاً وبنفس القدر من الجدارة كالذى استحقته عمارة يعقوبيان. فرحت عند انتهائى من قراءتها لأكثر من سبب، فقد أكدت لى هذه القراءة أن لدينا بالفعل أديبًا كبيرًا وموهوبًا، وظهر أن عمارة يعقوبيان ليست ظاهرة منفردة لا تتكرر، بل إن من الممكن أن تتكرر المرة بعد المرة. فى الرواية الجديدة (شيكاجو) كل مزايا الرواية السابقة: التشويق الذى يبدأ من أول صفحة ويستمر إلى آخر صفحة، أسلوب الكتابة السلس والسريع الذى يصيب الهدف باستمرار بلا تثاقل أو تسكع، الرسم الواضح والمتسق للشخصيات، اللغة العربية الراقية دون تكلف أو تعمد الإغراب، وقبل كل شيء وفوق كل شيء، نُبل المعني، إذ لا جدوى فى رأيى من رواية مهما كانت درجة تشويقها وإتقانها إذا لم تكن نبيلة المقصد، وإذا كان المقصد تافهًا أو حقيرًا قضى على ما قد يكون للمهارة والشطارة من أثر فى نفس القارئhttp://www.titaniumstores.com/dp/9770...,"Alaa Al Aswany, علاء الأسواني"
2418,4509,2666,"A cuatro profesores de literatura, Pelletier, Morini, Espinoza y Norton, los une su fascinación por la obra de Beno von Archimboldi, un enigmático escritor alemán cuyo prestigio crece en todo el mundo. La complicidad se vuelve vodevil intelectual y desemboca en un peregrinaje a Santa Teresa (trasunto de Ciudad Juárez), donde hay quien dice que Archimboldi ha sido visto. Ya allí, Pelletier y Espinoza se enteran de que la ciudad es desde años atrás escenario de una larga cadena de crímenes: en los vertederos aparecen cadáveres de mujeres con señales de haber sido violadas y torturadas. Es el primer asomo de la novela a sus procelosos caudales, repletos de personajes memorables cuyas historias, a caballo entre la risa y el horror, abarcan dos continentes e incluyen un vertiginoso travelling por la historia europea del siglo XX. 2666 confirma el veredicto de Susan Sontag: ""el más influyente y admirado novelista en lengua española de su generación. Su muerte, a los cincuenta años, es una gran pérdida 

In [25]:
df_books[df_books.book_id.isin(rec_als)][['book_id', 'original_title', 'book_desc', 'authors']]

,book_id,original_title,book_desc,authors
50,61,The Girl on the Train,"Rachel catches the same commuter train every morning. She knows it will wait at the same signal each time, overlooking a row of back gardens. She's even started to feel like she knows the people who live in one of the houses. 'Jess and Jason', she calls them. Their life - as she sees it - is perfect. If only Rachel could be that happy. And then she sees something shocking. It's only a minute until the train moves on, but it's enough. Now everything's changed. Now Rachel has a chance to become a part of the lives she's only watched from afar. Now they'll see; she's much more than just the girl on the train...",Paula Hawkins
125,163,The Lost Hero,"Jason has a problem.He doesn’t remember anything before waking up in a bus full of kids on a field trip. Apparently he has a girlfriend named Piper, and a best friend named Leo. They’re all students at a boarding school for “bad kids.” What did Jason do to end up here? And where is here, exactly?Piper has a secret.Her father has been missing for three days, ever since she had that terrifying nightmare about his being in trouble. Piper doesn’t understand her dream, or why her boyfriend suddenly doesn’t recognize her. When a freak storm hits during the school trip, unleashing strange creatures and whisking her, Jason, and Leo away to someplace called Camp Half-Blood, she has a feeling she’s going to find out.Leo has a way with tools.When he sees his cabin at Camp Half-Blood, filled with power tools and machine parts, he feels right at home. But there’s weird stuff, too—like the curse everyone keeps talking about, and some camper who's gone missing. Weirdest of all, his bunkmates insist that each of them—including Leo—is related to a god. Does this have anything to do with Jason's amnesia, or the fact that Leo keeps seeing ghosts?",Rick Riordan
222,294,The Son of Neptune,"Percy is confused. When he awoke from his long sleep, he didn't know much more than his name. His brain fuzz is lingering, even after the wolf Lupa told him he is a demigod and trained him to fight with the pen/sword in his pocket. Somehow Percy manages to make it to a camp for half-bloods, despite the fact that he has to keep killing monsters along the way. But the camp doesn't ring any bells with him. The only thing he can recall from his past is another name: Annabeth. Hazel is supposed to be dead. When she lived before, she didn't do a very good job of it. Sure, she was an obedient daughter, even when her mother was possessed by greed. But that was the problem—when the Voice took over her mother and commanded Hazel to use her ""gift"" for an evil purpose, Hazel couldn't say no. Now because of her mistake, the future of the world is at risk. Hazel wishes she could ride away from it all on the stallion that appears in her dreams. Frank is a klutz. His grandmother says he is descended from heroes and can be anything he wants to be, but he doesn't see it. He doesn't even know who his father is. He keeps hoping Apollo will claim him, because the only thing he is good at is archery—although not good enough to win camp war games. His bulky physique makes him feel like an ox, especially in front of Hazel, his closest friend at camp. He trusts her completely—enough to share the secret he holds close to his heart. Beginning at the ""other"" camp for half-bloods and extending as far as the land beyond the gods, this breathtaking second installment of the Heroes of Olympus series introduces new demigods, revives fearsome monsters, and features other remarkable creatures, all destined to play a part in the Prophesy of Seven.",Rick Riordan
692,980,Moon Called,"Mercedes Thompson, aka Mercy, is a talented Volkswagen mechanic living in the Tri-Cities area of Washington. She also happens to be a walker, a magical being with the power to shift into a coyote at will. Mercy's next-door neighbor is a werewolf. Her former boss is a gremlin. And she's fixing a bus for a 

## Evaluación de interacciones

In [26]:
# Métricas de evaluación 
# Obtenido de https://gist.github.com/bwhite/3726239

def precision_at_k(r, k):
    assert k >= 1
    r = np.asarray(r)[:k] != 0
    if r.size != k:
        raise ValueError('Relevance score length < k')
    return np.mean(r)

def average_precision(r):
    r = np.asarray(r) != 0
    out = [precision_at_k(r, k + 1) for k in range(r.size) if r[k]]
    if not out:
        return 0.
    return np.mean(out)

def mean_average_precision(rs):
    return np.mean([average_precision(r) for r in rs])
  
def dcg_at_k(r, k):
    r = np.asfarray(r)[:k]
    if r.size:
        return np.sum(np.subtract(np.power(2, r), 1) / np.log2(np.arange(2, r.size + 2)))
    return 0.


def ndcg_at_k(r, k):
    idcg = dcg_at_k(sorted(r, reverse=True), k)

    if not idcg:
        return 0.
    return dcg_at_k(r, k) / idcg

### Contenido

In [31]:
start = time.time()

mean_map = 0.
mean_ndcg = 0.

embeddings = pca20_bert_featmat
topk = 10 

for i, u in enumerate(user_interactions_test.keys()):
    
    print(i, end= '\r')
    
    rec = recommend(pca20_bert_large_featmat, user_id = u, topk=topk)
    rel_vector = [np.isin(user_interactions_test[u], rec, assume_unique=True).astype(int)]
    mean_map += mean_average_precision(rel_vector)
    mean_ndcg += ndcg_at_k(rel_vector, topk)

mean_map /= len(user_interactions_test)
mean_ndcg /= len(user_interactions_test)

time_taken = time.time() - start

In [32]:
print('MAP ',mean_map)
print('ndcg@10' ,mean_ndcg)
print('tiempo de ejecucion {0:.2f} segs'.format(time_taken))

MAP  0.026123015873015874
ndcg@10 0.07
tiempo de ejecucion 16.76 segs


### ALS

In [29]:
start = time.time()

mean_map = 0.
mean_ndcg = 0.

embeddings = pca20_bert_featmat
topk = 10 

for i, u in enumerate(user_interactions_test.keys()):
    
    print(i, end= '\r')
    
    recommendations = model_als.recommend(userid=user_ids[965], user_items=user_item_matrix[user_ids[965]], N=10)
    rec = [idx2bookid[r] for r in recommendations[0]]
    rel_vector = [np.isin(user_interactions_test[u], rec, assume_unique=True).astype(int)]
    mean_map += mean_average_precision(rel_vector)
    mean_ndcg += ndcg_at_k(rel_vector, topk)

mean_map /= len(user_interactions_test)
mean_ndcg /= len(user_interactions_test)

time_taken = time.time() - start

In [30]:
print('MAP ',mean_map)
print('ndcg@10' ,mean_ndcg)
print('tiempo de ejecucion {0:.2f} segs'.format(time_taken))

MAP  0.008333333333333333
ndcg@10 0.03
tiempo de ejecucion 0.09 segs
